# Libs

In [ ]:
import collections
import numbers
import operator
import functools
import bisect
import threading as th
import collections
import time
import ssl
import websocket
import json
import contextlib
import os
import pathlib
import shutil
import warnings
import re

In [ ]:
import math
import numpy as np
import scipy as sp
import pandas as pd

pd.set_option("max_rows", 128)
pd.set_option("max_columns", None)
pd.set_option("max_colwidth", 1024)

import matplotlib.pyplot as plt
from matplotlib import animation
from mpl_toolkits.mplot3d.axes3d import Axes3D
import seaborn as sns
# import plotly
# import plotly.offline as po
# import plotly.graph_objs as pg
# import plotly.figure_factory as pf

In [ ]:
if True:
    %matplotlib inline
    plt.rcParams["figure.figsize"] = (8.0, 6.0)
    plt.rcParams["animation.html"] = "html5"
elif False:
    %matplotlib notebook
sns.set()
sns.set_style("whitegrid")
# po.init_notebook_mode(connected=True)

# import cufflinks as cf
# cf.set_config_file(offline=True)

# from sklearn import datasets
from sklearn import (
    preprocessing,
    pipeline,
    metrics,
    dummy
)

In [ ]:
from sklearn import (
    linear_model,
    ensemble,
    neural_network,
    tree,
    svm
)
from sklearn import (
    neighbors,
    manifold,
    decomposition,
    cluster
)
from sklearn import (
    model_selection,
    utils
)

# import eli5
# import lime
# import lime.lime_tabular
# import shap

import torch
torch.set_default_tensor_type('torch.FloatTensor')

# import xgboost as xgb
# import gensim

import PIL
import cv2
import torchvision

In [ ]:
from torch import (
    nn,
    optim,
    utils,
)
from PIL import (
    Image,
    ImageEnhance,
)
from torchvision import (
    datasets,
    transforms,
    models,
)

# Sandbox

In [ ]:
# nbsync notebook.ipynb destdir

In [ ]:
src = "playground.ipynb"
# with open(src, "r", encoding="utf-8") as file:
#     content = json.load(file)

In [ ]:
# type(content["cells"])

In [ ]:
# cells = content["cells"]
# len(cells)

In [ ]:
# cells[5]

In [ ]:
# export nbsync.py:8
def _extract_relevant_cells(filename):
    with open(filename, "r", encoding="utf-8") as file:
        regex = re.compile(REGEX)
        return [
            cell
            for cell in json.load(file)["cells"]
            if (
                cell["cell_type"] == "code"
                and cell["source"]
                and regex.match(cell["source"][0]) is not None
            )
        ]

In [ ]:
cells = _extract_relevant_cells(src)
# cells

In [ ]:
cells[-5]

In [ ]:
# print("\n\n".join("".join(cell["source"]) for cell in cells))

In [ ]:
# export nbsync.py:1
import argparse
import logging
import pathlib
import re
import json

In [ ]:
cells = _extract_relevant_cells(src)
files = _sort_cells_by_files(cells)
for filename, cells in files.items():
    content = _assemble_file(cells, True, True)
    dest = pathlib.Path("/home/ihor/Dropbox/Documents/Toolbox/") / filename
#     with open(dest, "w") as file:
#         file.write(content)

In [ ]:
# print(content)
logging.BASIC_FORMAT

In [ ]:
# export nbsync.py:2
def main():
    """Entry point of the application."""
    args = _Parser().specify_args().parse_args()
    logging.basicConfig(level="INFO", format="%(levelname)s - %(message)s")
    logger = logging.getLogger(__name__)
    logger.info("processing %s", args.notebook)
    cells = _extract_relevant_cells(args.notebook)
    files = _sort_cells_by_files(cells)
    for filename, cells in files.items():
        if args.dest.is_dir():
            dest = args.dest / filename
        else:
            if args.dest.name == filename.name:
                dest = args.dest
            else:
                logger.info(
                    "only %s chosen, skippin' %s", args.dest, filename
                )
                continue
        if args.interactive:
            if not input(
                "Processing %s, enter y to proceed: " % dest
            ) == "y":
                logger.info("skippin' %s", dest)
                continue
        # TODO: refactor out filename
        content = _assemble_file(
            cells, args.shebang, args.main, args.docstring, filename
        )
        if dest.exists() and not args.force:
            logger.info("%s exists, skippin'", dest)
            continue
        with open(dest, "w") as file:
            file.write(content)
        logger.info("written %s", dest)

In [ ]:
# export nbsync.py:3
class _Parser(argparse.ArgumentParser):

    def specify_args(self):
        self.add_argument("notebook")
        class _ConvertToPath(argparse.Action):
            """"""

            def __call__(self, _, args, arg, ___):
                result = pathlib.Path(arg)
                setattr(args, self.dest, result)
        self.add_argument(
            "dest", help="directory of file", action=_ConvertToPath
        )
        self.add_argument("-f", "--force", action="store_true")
        self.add_argument("-s", "--shebang", action="store_true")
        self.add_argument("-m", "--main", action="store_true")
        self.add_argument("-i", "--interactive", action="store_true")
        self.add_argument("-d", "--docstring", help=(
            "override module docstring.  Worth noting that: 1. \"does the "
            "right thing\" for one-liners only, 2. applies to all targets, 3. "
            "sets cell number to 0"
        ))
        return self

In [ ]:
# export nbsync.py:4
def _merge(upper, *args):
    return (
        "# -*- coding: utf-8 -*-\n"
        + "# GENERATED BY NBSYNC.  MODIFY AT YOUR OWN RISK.\n"
        + "\n\n\n".join(
            "".join(cell["source"][1:]) for cell in upper
        )
        + "\n"
    )

In [ ]:
# export nbsync.py:5
def _assemble_file(cells, shebang, main, docstring=None, filename=None):
    upper = _find_upper_level_exports(cells)
    if docstring is not None:
        upper = _set_docstring(upper, docstring, filename)
    content = _merge(_reorder(upper))
    if shebang:
        content = "#!/usr/bin/env python3.6\n" + content
    if main:
        content += '\n\nif __name__ == "__main__":\n    main()\n'
    return content

In [ ]:
# export nbsync.py:5.125
def _find_docstring_cell(cells):
    return next((cell for cell in cells if cell["source"][1].startswith('"""')), None)

In [ ]:
# export nbsync.py:5.25
def _set_docstring(cells, docstring, filename):
    docstring_cell = _find_docstring_cell(cells)
    if docstring_cell is None:
        return [{"source": [
            "# export %s:0\n" % filename,
            '"""%s"""' % docstring
        ]}] + cells
    docstring_cell["source"] = (
        docstring_cell["source"][:1] + ['"""%s"""' % docstring]
    )
    return cells

In [ ]:
# export nbsync.py:5.5
def _reorder(cells):
    regex = re.compile(REGEX)
    def _define_number(cell):
        number = regex.match(cell["source"][0]).group("number")
        if number is None:
            return None
        return float(number)
    if all(_define_number(cell) is not None for cell in cells):
        return sorted((cell for cell in cells), key=_define_number)
    # Hopefully
    docstring = _find_docstring_cell(cells)
    imports = [cell for cell in cells if cell["source"][1].startswith('import') or cell["source"][1].startswith('from')]
    constants = [cell for cell in cells if cell["source"][1].split()[0].isupper()]
    main = next((cell for cell in cells if next(line for line in cell["source"][1:] if not line.startswith("@")).startswith("def main")), None)
    public_classes = [
        cell for cell in cells
        if (
            next(line for line in cell["source"][1:] if not line.startswith("@")).startswith("class ")
            and not next(line for line in cell["source"][1:] if not line.startswith("@")).startswith("class _")
        )
    ]
    public_functions = [
        cell for cell in cells
        if (
            next(line for line in cell["source"][1:] if not line.startswith("@")).startswith("def ")
            and not next(line for line in cell["source"][1:] if not line.startswith("@")).startswith("def _")
            and not next(line for line in cell["source"][1:] if not line.startswith("@")).startswith("def main")
        )
    ]
    private_classes = [
        cell for cell in cells
        if next(line for line in cell["source"][1:] if not line.startswith("@")).startswith("class _")
    ]
    private_functions = [
        cell for cell in cells
        if next(line for line in cell["source"][1:] if not line.startswith("@")).startswith("def _")
    ]
    ordered = []
    if docstring is not None:
        ordered.append(docstring)
    ordered.extend(imports)
    ordered.extend(constants)
    if main is not None:
        ordered.append(main)
    ordered.extend([*public_classes, *public_functions, *private_classes, *private_functions])
    ordered.extend([cell for cell in cells if cell not in ordered])
    return ordered

In [ ]:
# export nbsync.py:6
def _sort_cells_by_files(cells):
    files = {}
    regex = re.compile(REGEX)
    def _define_filename(cell):
        return pathlib.Path(regex.match(cell["source"][0]).group("filename"))
    for cell in cells:
        files.setdefault(_define_filename(cell), []).append(cell)
    return files

In [ ]:
_sort_cells_by_files(cells)

In [ ]:
# export nbsync.py:1.5
REGEX = "^#[ ]*export[ ]*(?P<filename>[\/A-Za-z0-9._-]+.py)(:(?P<number>[0-9.]+))?$"

In [ ]:
# export nbsync.py:7
def _find_upper_level_exports(cells):
    regex = re.compile(REGEX)
    return [cell for cell in cells if regex.match(cell["source"][0]) is not None]

In [ ]:
# print(
#     "".join(_define_header(cells))
# )

In [ ]:
_find_upper_level_exports(cells)

In [ ]:
# export nbsync.py:0
"""Simple tool to facilitate explorative programming."""

In [ ]:
# export nbclean.py
"""Another simple tool to facilitate explorative programming.

Should be merged with nbsync.
"""

In [ ]:
# export nbclean.py
import json
import collections
import argparse
import logging

In [ ]:
# export nbclean.py
def main():
    """Entry point of the application."""
    args = _Parser().specify_args().parse_args()
    logging.basicConfig(level="INFO", format="%(levelname)s - %(message)s")
    logger = logging.getLogger(__name__)
    logger.info("processing %s", args.notebook)
    with open(args.notebook) as file:
        obj = json.load(file, object_hook=collections.OrderedDict)
    for cell in obj["cells"]:
        if cell["cell_type"] == "markdown":
            continue
        cell["execution_count"] = None
        if args.outputs:
            # Remove outputs completely
            if "outputs" in cell:
                cell["outputs"] = []
        else:
            # Clean outputs' execution_count
            outputs = cell.get("outputs", [])
            if outputs:
                for output in outputs:
                    output["execution_count"] = None
    if not args.force:
        logger.info("not --force, not overriding %s", args.notebook)
        return
    with open(args.notebook, "w") as file:
        json.dump(obj, file, indent=1, ensure_ascii=False)
        file.write("\n")

In [ ]:
# export nbclean.py
class _Parser(argparse.ArgumentParser):

    def specify_args(self):
        self.add_argument("notebook")
        self.add_argument("-f", "--force", action="store_true")
        self.add_argument("-o", "--outputs", action="store_true")
        return self